In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git

In [6]:
from src.fn17 import load_dataset_hf
from src.train import train
from src.evaluate import compute_metrics

In [ ]:
dataset = load_dataset_hf(flatten=True)
dataset

## Train

In [ ]:
OUT_DIR = './models/'
CHECKPOINT = 'bert-base-cased'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-5

train(pretrained_model=CHECKPOINT, dataset=dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)


## Test

## Predict